# $\color{red} {\text{Coordinate Descent}}$

> ## $\color{cyan}{\text{What is Coordinate Descent?}}$

>> **Coordinate descent** is an optimization algorithm that successively minimizes along coordinate directions to find the minimum of a function. At each iteration, the algorithm determines a coordinate or coordinate block via a coordinate selection rule, then exactly or inexactly minimizes over the corresponding coordinate hyperplane while fixing all other coordinates or coordinate blocks. A line search along the coordinate direction can be performed at the current iterate to determine the appropriate step size. Coordinate descent is applicable in both differentiable and derivative-free contexts.

> ## $\color{cyan}{\text{What types are done here?}}$

>> * random coordinate descent
>> * cyclic coordinate descent
>> * adaptive coordinate descent

In [ ]:
#imports
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn import preprocessing
from scipy.optimize import fmin
from random import randint
import pandas as pd
import numpy as np
import matplotlib.pyplot as pp
from google.colab import files

In [ ]:
def lf(wi, x, y, w,idx = 'N'):
  loss = 0
  if idx != 'N':
    w[idx] = wi
  for i in range(len(y)):
    if y[i] == 1:
      loss += np.log(1.0 + np.exp(-np.dot(w,x[i,:])))
    else:
      loss += np.log(1.0 + np.exp(np.dot(w,x[i,:])))
  return loss

In [ ]:
# successfully replicated score function now with w as parameter
def test_score(x, y, w):
  pred_y = np.zeros(len(y))
  ll = 0
  err = 0
  for i in range(len(y)):
    one = 1.0/(1.0 + np.exp(-np.dot(w,x[i,:])))
    zero = 1.0 - one
    pred_y[i] = 1 if one > zero else 0
    if pred_y[i] != y[i]:
      err = err + 1
  return (1 - err/len(y))

In [ ]:
# gradient function - V2
# will calculate gradient for all values at once
def gradient(x,y,w):
  # returns a (x.length,1) array of the P(y=1) for each x 
  pred = 1.0/(1.0 + np.exp(-np.dot(x,w)))
  pred = np.reshape(pred,(104,1))
  grad = np.dot(x.T,np.subtract(pred, y))
  return grad

In [ ]:
# how to pick idx to minimize
def adaptive_idx(x,y,w):
  grad = gradient(x,y,w)
  max_i = np.argmax(abs(grad))
  return max_i

In [ ]:
# minimize the loss function
# might redo so i have the learning rate
def update_wi(x, y, w, wi, index):
  updated_wi = fmin(lf,args=(x,y,w,index),x0 = wi, disp = False)
  return updated_wi

In [ ]:
# data preprocessing
df = load_wine(as_frame = True)
# concatenating horizontally
data = pd.concat([df.data, df.target], axis = 1)
# slicing the data so only those of class 0 and 1 are included (shuffling included)
data = data.loc[data['target'] != 2].sample(frac = 1)
# get training and testing datasets
train, test = train_test_split(data,test_size = 0.2)
train_x = np.split(train,[13,14],axis = 1)[0].to_numpy()
train_y = np.split(train,[13,14],axis = 1)[1].to_numpy()
test_x = np.split(test,[13,14],axis = 1)[0].to_numpy()
test_y = np.split(test,[13,14],axis = 1)[1].to_numpy()

In [ ]:
# baseline logistic regression - test
from sklearn.linear_model import LogisticRegression
#need to have penalty or else will overfit
lr = LogisticRegression(solver='lbfgs',max_iter=1000000,C=100000)
lr.fit(train_x, np.ravel(train_y))
score_star = lr.score(test_x,np.ravel(test_y))
print(score_star)
lr_pred_prob_test = lr.predict_proba(test_x)
lr_pred_prob_train = lr.predict_proba(train_x)
w_star = lr.coef_
b_star = lr.intercept_
# combine the w* and b* to get w'*
w_p_star = np.append(b_star, w_star)
print(w_p_star)
# calculate the final loss (log_loss function + maybe self-implemented function)
lr_loss_test = log_loss(np.ravel(test_y), lr_pred_prob_test, normalize=False)
lr_loss_train = log_loss(np.ravel(train_y), lr_pred_prob_train, normalize=False)
print(lr.n_iter_)
print(lr_loss_test)
print(lr_loss_train)

In [ ]:
# add intercept column
train_int = np.ones((len(train_x),1))
test_int = np.ones((len(test_x),1))
train_x_b = np.hstack((train_int,train_x))
test_x_b = np.hstack((test_int,test_x))

In [ ]:
# normalize the data
train_x_norm = preprocessing.normalize(train_x_b, norm = 'l2')
test_x_norm = preprocessing.normalize(test_x_b, norm = 'l2')

In [ ]:
#random coordinate descent
rcd_loss = []
w_rcd = np.random.rand(14)
rcd_ct = 0
cur_rcd_loss = 10000
max_iter = 10000
while cur_rcd_loss > 1 and rcd_ct < max_iter:
   idx = randint(0,13)
   updated_wi = update_wi(train_x_norm,train_y,w_rcd,w_rcd[idx],idx)[0]
   w_rcd[idx] = updated_wi
   cur_rcd_loss = lf(1,train_x_norm,train_y,w_rcd)
   rcd_loss.append(cur_rcd_loss)
   print(cur_rcd_loss)
   rcd_ct += 1

In [ ]:
print(test_score(test_x_b,test_y,w_rcd))
print(lf(1,test_x_norm,test_y,w_rcd))
print(w_rcd)
print(rcd_ct)

In [ ]:
#cyclic coordinate descent
ccd_loss = []
w_ccd = np.random.rand(14)
ccd_ct = 0
cur_ccd_loss = 10000
idx = 0
max_iter = 10000
cur_iter = 0
while cur_ccd_loss > 1 and ccd_ct < max_iter:
   idx = idx + 1 if idx + 1 < 14 else 0
   updated_wi = update_wi(train_x_norm,train_y,w_ccd,w_ccd[idx],idx)[0]
   w_ccd[idx] = updated_wi
   cur_ccd_loss = lf(1,train_x_norm,train_y,w_ccd)
   ccd_loss.append(cur_ccd_loss)
   print(cur_ccd_loss)
   ccd_ct += 1

In [ ]:
print(test_score(test_x_b,test_y,w_ccd))
print(lf(1,test_x_norm,test_y,w_ccd))
print(w_ccd)
print(ccd_ct)

In [ ]:
#adaptive coordinate descent
acd_loss = []
acd_dict = {}
w_acd = np.random.rand(14)
cur_acd_loss = 10000
acd_ct = 0
max_iter = 10000
while cur_acd_loss > 1 and acd_ct < max_iter:
  idx = adaptive_idx(train_x_norm,train_y,w_acd)
  if (idx in acd_dict):
    acd_dict[idx] += 1
  else:
    acd_dict[idx] = 1
  updated_wi = update_wi(train_x_norm, train_y,w_acd,w_acd[idx],idx)[0]
  w_acd[idx] = updated_wi
  cur_acd_loss = lf(1,train_x_norm,train_y,w_acd)
  acd_loss.append(cur_acd_loss)
  print(cur_acd_loss)
  acd_ct += 1

In [ ]:
print(test_score(test_x_b,test_y,w_acd))
print(lf(1,test_x_norm,test_y,w_acd))
print(w_acd)
print(acd_ct)
print(acd_dict)

In [ ]:
#create graph of each side by side
pp.xlabel('Number of iterations')
pp.ylabel('Training loss')
pp.xlim([0, 4000])
pp.plot(rcd_loss)
pp.plot(ccd_loss)
pp.plot(acd_loss)
pp.legend(['RCD','CCD','ACD'])
pp.show()